In [ ]:
# For google colab
#!pip install erddap-python

In [1]:
import matplotlib.pyplot as plt
from erddapClient import ERDDAP_Griddap
import pandas as pd
from pathlib import Path
import imageio
from tqdm import tqdm
import xarray as xr
import warnings
from loguru import logger
from glob import glob

import matplotlib

matplotlib.use("Agg")

warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
@logger.catch
def get_data(server, datasetID, vars, time, nc_output_file):
    subset = (
        ERDDAP_Griddap(server, datasetID)
        .setResultVariables(list(vars.keys()))
        .setSubset(
            time=time,
            depth=slice(0.5000003, 0.5000003),
            gridY=slice(600, 897),
            gridX=slice(100, 300),
        )
        .getxArray()
    )
    # save dataset
    # Drop some atttributes that fails to be encoded
    for var in subset.variables:
        subset[var].attrs.pop("_evenlySpaced", None)
        subset[var].attrs.pop("actual_range", None)
        subset[var].attrs.pop("units", None)
        subset[var].attrs.pop("calendar", None)
    subset.to_netcdf(nc_output_file)
    return subset

In [3]:
server = "https://salishsea.eos.ubc.ca/erddap"
datasetID = "ubcSSg3DPhysicsFields1hV21-11"
vars = {
    "temperature": {"vmin": 5, "vmax": 25, "cmap": "inferno"},
    "salinity": {"vmin": 20, "vmax": 32, "cmap": "viridis"},
}
get_data(server, datasetID, vars, "2021-01-01T00:30:00Z", "test.nc")

<xarray.Dataset>
Dimensions:      (time: 1, depth: 1, gridY: 298, gridX: 201)
Coordinates:
  * time         (time) object 2021-01-01 00:30:00
  * depth        (depth) float64 0.5
  * gridY        (gridY) float64 600.0 601.0 602.0 603.0 ... 895.0 896.0 897.0
  * gridX        (gridX) float64 100.0 101.0 102.0 103.0 ... 298.0 299.0 300.0
Data variables:
    temperature  (time, depth, gridY, gridX) float32 ...
    salinity     (time, depth, gridY, gridX) float32 ...
Attributes: (12/25)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2024-02-14T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2024-Feb-14 17:20:55 GMT
    title:                     Green, Salish Sea, 3d Physics Fields, Hourly, ...
    uuid:                      3cd684be-5e8f-400a-96ac-05e0ad258ae4

In [4]:
@logger.catch
def get_frame(time, vars, output, server, datasetID):
    """Get water properties frames"""
    nc_output_file = Path(output.format(time=time, var="-".join(vars.keys())) + ".nc")
    if nc_output_file.exists():
        subset = xr.open_dataset(nc_output_file)
    else:
        subset = get_data(server, datasetID, vars, time, nc_output_file)

    for var, attrs in vars.items():
        _, ax = plt.subplots()
        subset[var].plot(ax=ax, **attrs)
        plt.savefig(output.format(time=time, var=var))

## Time Range of Interest


In [2]:
time_frames = pd.date_range(
    start=pd.to_datetime("2016-07-25T00:30:00Z"),
    end=pd.to_datetime("2016-08-05T00:30:00Z"),
    freq=pd.Timedelta("1h"),
)
Path("output").mkdir(parents=True, exist_ok=True)

## Water Properties 

In [6]:
# Get temperature,salinity frames
server = "https://salishsea.eos.ubc.ca/erddap"
datasetID = "ubcSSg3DPhysicsFields1hV21-11"

vars = {
    "temperature": {"vmin": 5, "vmax": 25, "cmap": "inferno"},
    "salinity": {"vmin": 20, "vmax": 32, "cmap": "viridis"},
}

data = []
for time in tqdm(time_frames, desc="Generate frames", unit="time_stamp"):
    get_frame(
        time.isoformat().replace("+00:00", "Z"),
        vars,
        "output/{var}_{time}",
        server,
        datasetID,
    )

Generate frames:   0%|          | 0/265 [00:00<?, ?time_stamp/s]

Generate frames:   4%|▍         | 10/265 [00:02<00:51,  4.93time_stamp/s]/var/folders/bv/k82zg3_x1fg00tjywdqys88h0000gn/T/ipykernel_45947/2377147092.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  _, ax = plt.subplots()
Generate frames: 100%|██████████| 265/265 [00:57<00:00,  4.60time_stamp/s]


## Load Current data

In [7]:
subset = (
    ERDDAP_Griddap("https://salishsea.eos.ubc.ca/erddap", "ubcSSg3DuGridFields1hV21-11")
    .setResultVariables("uVelocity")
    .setSubset(
        time=time,
        depth=slice(0.5000003, 0.5000003),
        gridY=slice(600, 897),
        gridX=slice(100, 300),
    )
    .getxArray()
)

In [8]:
server = "https://salishsea.eos.ubc.ca/erddap"
datasetIDs = {
    "uVelocity": "ubcSSg3DuGridFields1hV21-11",
    "vVelocity": "ubcSSg3DvGridFields1hV21-11",
}

for time in tqdm(time_frames, desc="Generate frames", unit="time_stamp"):
    for var, datasetID in datasetIDs.items():
        get_frame(
            time.isoformat().replace("+00:00", "Z"),
            {var: {"vmin": -1, "vmax": 1, "cmap": "coolwarm"}},
            "output/{var}_{time}",
            server,
            datasetID,
        )

Generate frames: 100%|██████████| 265/265 [01:02<00:00,  4.25time_stamp/s]


## Load Sentry Shoal

In [9]:
## Load Sentry Shoal data

## Generate Combined figures

In [3]:
# Generate each frames
def make_combined_figure(timestamp_str):
    try:
        ds_water = xr.open_dataset(f"output/temperature-salinity_{timestamp_str}.nc")
        ds_u = xr.open_dataset(f"output/uVelocity_{timestamp_str}.nc")
        ds_v = xr.open_dataset(f"output/vVelocity_{timestamp_str}.nc")
    except:
        logger.error(f"Could not open files for {timestamp_str}")
        return
    
    for var in ["temperature", "salinity"]:
        ds_water[var] = ds_water[var].where(ds_water[var] > 0)
    
    ds_u["uVelocity"] = ds_u["uVelocity"].where(ds_u["uVelocity"] != 0)
    ds_v["vVelocity"] = ds_v["vVelocity"].where(ds_v["vVelocity"] != 0)

    plt.close("all")

    def get_cmap(cmap_name):
        cmap = plt.get_cmap(cmap_name)
        cmap.set_under(color="white")
        return cmap

    cur_spd = (ds_u.uVelocity**2 + ds_v.vVelocity**2) ** (1 / 2)

    col = 2
    row = 2
    fig, axs = plt.subplots(col, row, figsize=(8, 5))
    fig.suptitle(f"SalishSeaCast Model Output: {timestamp_str}")
    ds_water.temperature.plot(ax=axs[0, 0], cmap="turbo", vmin=12, vmax=22)
    ds_water.salinity.plot(ax=axs[0, 1], cmap="turbo", vmin=24, vmax=30)
    ((ds_u.uVelocity**2 + ds_v.vVelocity**2) ** (1 / 2)).plot(
        ax=axs[1, 0],
        cmap=get_cmap("turbo"),
        vmin=0.01,
        vmax=1,
        cbar_kwargs={"label": "Current Speed (m/s)"},
    )
    plt.quiver(
        ds_u.gridX,
        ds_u.gridY,
        ds_u.uVelocity.isel(time=0, depth=0),
        ds_v.vVelocity.isel(time=0, depth=0),
        cur_spd.where(cur_spd > 0.1),
        scale=100,
        cmap="turbo",
    )

    # ds_v.vVelocity.plot(ax=axs[1, 1], cmap='coolwarm', vmin=-1, vmax=1)

    for i in range(col):
        for j in range(row):
            axs[i, j].set_title(None)
            axs[i, j].set_xlabel(None)
            axs[i, j].set_ylabel(None)
            axs[i, j].set_xticks([])
            axs[i, j].set_yticks([])

    plt.savefig(
        f"output/combined_{timestamp_str}.png",
        dpi=300,
        bbox_inches="tight",
        pad_inches=0.1,
    )

In [4]:
for timestamp in tqdm(time_frames, desc="Generate combined frames", unit="frame"):
    timestamp_str = timestamp.isoformat().replace("+00:00", "Z")
    make_combined_figure(timestamp_str)

Generate combined frames:   0%|          | 0/265 [00:00<?, ?frame/s]

Generate combined frames: 100%|██████████| 265/265 [04:02<00:00,  1.09frame/s]


In [5]:
def make_gif(files, output, fps):
    frames = []
    for file in tqdm(sorted(glob(files)), desc="Load frames", unit="frame"):
        image = imageio.v2.imread(file)
        frames.append(image)
    logger.info(f"Saving gif to {output}")
    imageio.mimsave(output, frames, fps=fps)
    logger.info(f"Done")

In [6]:
make_gif("output/combined_*.png", "../docs/NSOG_animation_20160725-20160805.gif", 4)

Load frames: 100%|██████████| 265/265 [00:14<00:00, 18.21frame/s]
2024-02-14 16:36:39.097 | INFO     | __main__:make_gif:6 - Saving gif to ../docs/NSOG_animation_20160725-20160805.gif
2024-02-14 16:36:56.715 | INFO     | __main__:make_gif:8 - Done


In [18]:
ds_temp = xr.open_dataset("output/temperature-salinity_2016-08-03T04:30:00Z.nc")

In [26]:
plt.close()
ds_temp['temperature'].where(ds_temp['temperature'] > 0).plot()
plt.savefig("temperature.png")

In [30]:
plt.close()
ds_temp = xr.open_dataset("output/vVelocity_2016-08-04T19:30:00Z.nc")
ds_temp['vVelocity'].where(ds_temp['vVelocity'] != 0).plot()
plt.savefig("vVelocity.png")
